### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만큼 데이터를 조절하기 위한 메타니즘
- Dataset : 사용 데이터를 기반으로 사용자정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size 만큼 피쳐와 타겟을 추출하여 전달

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
### ===> 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd


In [2]:
### ===> 데이터 준비
x_data = torch.IntTensor([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]])
y_data = torch.IntTensor([[20],[30],[40],[50],[60]])

print(f'x_data => {x_data.shape}, {x_data.ndim}D')
print(f'y_data => {y_data.shape}, {y_data.ndim}D')

x_data => torch.Size([5, 3]), 2D
y_data => torch.Size([5, 1]), 2D


[2] 데이터셋 생성 <hr>

- [2-1] TensorDataset 활용 : Dataset의 sub_class

In [4]:
# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [5]:
dataset = TensorDataset(x_data, y_data)
dataset

In [6]:
dataset.tensors

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [7]:
## __getitem__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20], dtype=torch.int32))

In [8]:
len(dataset)

5

- [2-2] 사용자정의 데이터셋 생성

In [12]:
### 데이터 준비
filename = '../data/text/iris2.csv'
irisDF = pd.read_csv(filename,header=None)
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
 4   4       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [15]:
irisNP = np.loadtxt(filename, delimiter=',', usecols=[0,1,2,3])
irisNP


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [16]:
## 데이터의 타입 체크
type(irisDF), type(irisNP)

(pandas.core.frame.DataFrame, numpy.ndarray)

In [17]:
if irisDF.__class__.__name__ == 'DataFrame':
    print('DF')
else:
    print("-------")

DF


In [18]:
isinstance(irisDF,pd.DataFrame), isinstance(irisNP, pd.DataFrame)

(True, True)

In [20]:
isinstance([10], list), isinstance({'A':22}, list)      # 인스턴스형태의 일치성 확인?

(True, False)

True

In [ ]:
### 사용자 정의 DataSet 클래스
# - 데이터의 Tensor 변환
class DLDataset(Dataset):
    
    # 초기화 함수 콜백함수(callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        self.feature = torch.tensor(x_data)
        self.target = torch.tensor(y_data)
    
    # 데이터셋의 갯수 체크 함수 콜백함수(checkback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터+라벨 반환 콜백함수(callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]